In [8]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px, html
from great_tables.data import islands
from pathlib import Path
from typing import Any
from ingestion.db_utils import load_report_params,fetch_vars_for_report, fetch_latest_table_data, insert_variable
from pathlib import Path
from datetime import datetime

# Get current year and subtract 1 for previous year
previous_year = datetime.now().year - 1

# Create a date object for December of the previous year
last_month_previous_year = datetime(previous_year, 12, 1)

# Format as "Mon YYYY"
formatted_date = last_month_previous_year.strftime("%b %Y")
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"


conn = sqlite3.connect(db_path)
cur = conn.cursor()
db_path = Path(conn.execute("PRAGMA database_list").fetchone()[2])
report = 'Quarterly_Report'
report_params = load_report_params(report_name=report, db_path=db_path)
report_quarter = report_params.get("overviewDate")
target_history = report_params.get("Budget_Impl")
table_colors = report_params.get("TABLE_COLORS")

BLUE = table_colors.get("BLUE", "#004A99") if table_colors else "#004A99"
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4") if table_colors else "#d6e6f4"
# GRID_CLR = table_colors.get("GRID_CLR", "#004A99") if table_colors else "#004A99"
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B") if table_colors else "#01244B"


outline_b = '2px'

# Summary_comm ###
report_variables = fetch_vars_for_report(report, db_path)
commitment_summary = report_variables.get("table_1a")
df_comm = pd.DataFrame(commitment_summary)

l1_comm = report_variables.get("table_1c")
df_l1 = pd.DataFrame(l1_comm)

# Filter for current year and specific Fund Sources

tot = df_comm.iloc[-1]         # last row *is* the total

ratio_pct = f"{tot['ratio_consumed_of_L1_and_L2_against_Commitment_Appropriations']*100:.2f}%"
avail_bln = f"{tot['Available_Commitment_Appropriations']/1e9:.2f}"

budget_impl = (
    "Consumption of commitment appropriations out of the total "
    "available credits (C1/E0-HEU)"
)
overview     = f"{ratio_pct} (out of € {avail_bln} bln)"
target_history_comm = target_history['row2_commit'] 

summary_comm = pd.DataFrame(
    {   
        'Indicator' : "Budget implementation",
        "Indicator_Description": [budget_impl],
        f'{report_quarter}': [overview],
        f"{formatted_date}" : f'{target_history_comm}',
        'Target' : '100%'
    }
)


# Summary_L1 ###
df_l1 = df_l1.loc[df_l1["Fund Source"] == "VOBU/EFTA/IAR2/2"]

total_l1 = df_l1['L1_Commitment_1']
total_l2 = df_l1['L2_Commitment_2']

total_l2 = -1*total_l2.iloc[-1]
total_l1 = total_l1.iloc[-1]
pct_L2_L1 = (total_l2/total_l1)*100

target_history_L1 = target_history['row3_L2/L1'] 

budget_impl = (
    "Consumption of Carried Forward Commitment Voted Credits (% L2 on "
    "L1) of previous year calls"
)
output_str = f"{pct_L2_L1:.0f} % (out of € {total_l1/1e9:.1f} bln)"


summary_L1 = pd.DataFrame(
    {   
        'Indicator' : "Budget implementation",
        "Indicator_Description": [budget_impl],
        f'{report_quarter}': f'{output_str}',
        f"{formatted_date}" : f'{target_history_L1}',
        'Target' : '100%'
    }
)

# Summary payments ###

p_h2020 = report_variables.get("table_2a_H2020")
df_p_h2020 = pd.DataFrame(p_h2020)
h2020_total  = df_p_h2020.iloc[-1]   


p_he = report_variables.get("table_2a_HE")
df_p_he = pd.DataFrame(p_he)
he_total = df_p_he.iloc[-1]

total_p_h2020 = h2020_total['Paid_Amount']
total_p_a_h2020 = h2020_total['Available_Payment_Appropriations']

total_p_he =  he_total['Paid_Amount']
total_p_a_he = he_total['Available_Payment_Appropriations']

total_p_appr = total_p_a_h2020 + total_p_a_he
total_p_cons = total_p_h2020 + total_p_he 

ratio_pct = f"{total_p_cons/total_p_appr*100:.2f}%"

payments_impl = (
    "Consumption of payment appropriations out of the total available "
    "credits (operational)"
    "(VOBU/EFTA - H2020 + HEU)"
)

output_str = f"{ratio_pct} (out of € {total_p_appr/1e9:.1f} bln)"
payments_target = target_history['row4_paym']

summary_payments = pd.DataFrame(
    {
        'Indicator' : "Budget implementation",
        "Indicator_Description": [payments_impl],
        f'{report_quarter}': f'{output_str}',
        f"{formatted_date}" : f'{payments_target}',
        'Target' : '100%'
    }
)

# Summary EARN ###

df_comm = fetch_latest_table_data(conn, "c0_commitments_summa", cutoff)
df_comm_earn = df_comm.loc[df_comm["Fund Source"] == "EARN/N"]

total_earn_appr = df_comm_earn['FR Accepted Amount'].sum()
total_earn_committed = -1* (df_comm_earn['FR Consumption by PO Amount'].sum())

ratio_comm_pct = f"{total_earn_committed/total_earn_appr*100:.2f}%"

earn_impl = (
    "Commitment appropriations concerning a third country contribution "
    "(EARN - H2020 + HEU)6"
)

output_str = f"{ratio_comm_pct} (out of € {total_earn_appr/1e9:.1f} bln)"
earn_consum_target = target_history['row5_EARN']

summary_earn = pd.DataFrame(
    {
        'Indicator' : "Budget implementation",
        "Indicator_Description": [earn_impl],
        f'{report_quarter}': f'{output_str}',
        f"{formatted_date}" : f'{earn_consum_target}',
        'Target' : '-'
    }
)

vacancy_Rate = report_params.get("Vacancy_Rate")

try:
    prev_value = vacancy_Rate['previous_year']
    cur_val = vacancy_Rate['current_year']
except KeyError as e:
    logging.error(f"Missing expected key in Vacancy_Rate: {e}")
    raise


vacancy_Rate = report_params.get("Vacancy_Rate", {})
prev_value = vacancy_Rate.get("previous_year", "N/A")
cur_val = vacancy_Rate.get("current_year", "N/A")

summary_vacancy = pd.DataFrame(
    {
        'Indicator': ["Resources"],
        'Indicator_Description': ["Vacancy rate for permanent posts"],
        f"{report_quarter}": [f"{cur_val}"],
        f"{formatted_date}": [f"{prev_value}"],
        'Target': ["2%"]
    }
)



combined_summary = pd.concat([summary_vacancy, summary_comm, summary_L1, summary_payments, summary_earn], ignore_index=True)


# Build GreatTables object
try:
    tbl_summary = (
    GT(
        combined_summary,
        groupname_col="Indicator",
        rowname_col="Indicator_Description"
    )

    .tab_stubhead("Indicator")
 
    # GENERAL FORMATTING
    # Table Outline
    .opt_table_outline(style = "solid", width = outline_b, color =  DARK_BLUE) 
    # Arial font
    .opt_table_font(font="Arial")
    # Header and stub styling
    .tab_style(
        style=[
            # style.borders(weight="1px", color=DARK_BLUE),
            style.fill(color=LIGHT_BLUE),
            style.text( weight="bold", align='center', size = 'medium'),
            style.css("text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;")
        ],
        locations=loc.column_labels()
    )
     .tab_style(
         style.text(size = 'small'),
        loc.stub(),
    )
     
    .tab_style(
    style.text(size = 'small'),
    loc.body()
    )
    
    # # Table borders
    .tab_options(
             container_width = "100%",
            column_labels_background_color = BLUE,
            row_group_background_color=LIGHT_BLUE,
            table_body_hlines_style="solid",
            table_body_vlines_style="solid",
   
        )

    )
except Exception as e:

    logging.error(f"Error building GreatTables object: {str(e)}")


# Store the table and data if save_to_db is True
try:
    insert_variable(
        report=report,
        module="GrantsModule",
        var="table_3c_PO_exceeding_FDI",
        value= combined_summary.to_dict(orient="records"),
        db_path=db_path,
        anchor="table_3c",
        gt_table=tbl_summary,
    )
except Exception as e:
        logging.error(f"Error storing table for table_3c: {str(e)}")

DEBUG:root:Fetching latest data for table_alias: c0_commitments_summa, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for c0_commitments_summa: [('2025-05-05T08:47:09.188161', 7)]
DEBUG:root:Checking upload_id: 7, uploaded_at: 2025-05-05T08:47:09.188161
DEBUG:root:Fetched 119 rows from c0_commitments_summa with upload_id 7
DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:Sending stats to Plausible: Props { browser: "chrome", browser_version: "", os: "windows", arch: "amd64", lang: "python", selenium_version: "4.32" }
DEBUG:selenium.webdriver.com

In [9]:
tbl_summary

GT(_tbl_data=               Indicator                              Indicator_Description  \
0              Resources                   Vacancy rate for permanent posts   
1  Budget implementation  Consumption of commitment appropriations out o...   
2  Budget implementation  Consumption of Carried Forward Commitment Vote...   
3  Budget implementation  Consumption of payment appropriations out of t...   
4  Budget implementation  Commitment appropriations concerning a third c...   

                  March 2025                      Dec 2024 Target  
0                       1.5%                         1.09%     2%  
1  0.77% (out of € 2.22 bln)      100% (out of € 2.24 bln)   100%  
2    39 % (out of € 1.6 bln)     100 % (out of € 2.0  bln)   100%  
3  24.26% (out of € 2.5 bln)    100 % (out of € 2.174 bln)   100%  
4  56.75% (out of € 0.4 bln)  94.21 % (out of € 0.984 bln)      -  , _body=<great_tables._gt_data.Body object at 0x0000021846816190>, _boxhead=Boxhead([ColInfo(var='Indicator', type=<ColInfoTypeEnum.row_group: 3>, column_label='Indicator', column_align='left', column_width=None), ColInfo(var='Indicator_Description', type=<ColInfoTypeEnum.stub: 2>, column_label='Indicator_Description', column_align='left', column_width=None), ColInfo(var='March 2025', type=<ColInfoTypeEnum.default: 1>, column_label='March 2025', column_align='left', column_width=None), ColInfo(var='Dec 2024', type=<ColInfoTypeEnum.default: 1>, column_label='Dec 2024', column_align='left', column_width=None), ColInfo(var='Target', type=<ColInfoTypeEnum.default: 1>, column_label='Target', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000021846443D90>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead='Indicator', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='March 2025', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color=None, font=None, size='medium', align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='Dec 2024', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color=None, font=None, size='medium', align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocColumnLabels(columns=None), grpname=None, colname='Target', rownum=None, colnum=None, styles=[CellStyleFill(color='#d6e6f4'), CellStyleText(color=None, font=None, size='medium', align='center', v_align=None, style=None, weight='bold', stretch=None, decorate=None, transform=None, whitespace=None), CellStyleCss(rule='text-align: center; vertical-align: middle; max-width:200px; line-height:1.2; font-size: smaller;')]), StyleInfo(locname=LocStub(rows=None), grpname=None, colname=None, rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size='small', align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocStub(rows=None), grpname=None, colname=None, rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size='small', align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocStub(rows=None), grpname=None, colname=None, rownum=2, colnum=None, styles=[CellStyleText(color=None, font=None, size='small', align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocStub(row

In [7]:
vacancy_Rate = report_params.get("Vacancy_Rate")

try:
    prev_value = vacancy_Rate['previous_year']
    cur_val = vacancy_Rate['current_year']
except KeyError as e:
    logging.error(f"Missing expected key in Vacancy_Rate: {e}")
    raise


vacancy_Rate = report_params.get("Vacancy_Rate", {})
prev_value = vacancy_Rate.get("previous_year", "N/A")
cur_val = vacancy_Rate.get("current_year", "N/A")

summary_vacancy = pd.DataFrame(
    {
        'Indicator': ["Resources"],
        'Indicator_Description': ["Vacancy rate for permanent posts"],
        f"{report_quarter}": [f"{cur_val}"],
        f"{formatted_date}": [f"{prev_value}"],
        'Target': ["2%"]
    }
)




summary_vacancy

,Indicator,Indicator_Description,March 2025,Dec 2024,Target
0,Resources,Vacancy rate for permanent posts,1.5%,1.09%,2%
